PYTHON
Starting with Helper Functions

In [62]:
import numpy as np

def MSD(traj, maxlag=None):
    """
    Calculate the Mean Squared Displacement (MSD) of a trajectory for all possible time lags.

    Parameters:
    - traj: numpy.ndarray, shape (N, dims+1)
      Input trajectory data where N is the number of frames and dims is the number of dimensions.
      The last column is typically used for time or frame index.
    
    - maxlag: int, optional
      The maximum time lag to calculate MSD for. If not provided, it's set to N-1.

    Returns:
    - msd: numpy.ndarray, shape (maxlag,)
      MSD values for each time lag from 1 to maxlag.
    """
    N = traj.shape[0]
    dims = traj.shape[1] - 1

    if maxlag is None:
        maxlag = N - 1

    msd = np.zeros(maxlag)

    pos = traj[:, :dims]
    
    for n in range(1, maxlag + 1):
        sqdisp = np.sum((pos[n:] - pos[:-n])**2, axis=1)
        msd[n-1] = np.mean(sqdisp)

    return msd


In [123]:
# import numpy as np

# def RadiusOfGyration(traj):
#     """
#     Calculate the Radius of Gyration tensor (T), eigenvectors (V), eigenvalues (lambda), 
#     and Radius of Gyration (R) of a trajectory.

#     Parameters:
#     - traj: numpy.ndarray, shape (N, dims)
#       Input trajectory data where N is the number of frames and dims is the number of dimensions.

#     Returns:
#     - T: numpy.ndarray, shape (dims, dims)
#       The Radius of Gyration tensor.
#     - V: numpy.ndarray, shape (dims, dims)
#       The eigenvectors of the Radius of Gyration tensor.
#     - lambda: numpy.ndarray, shape (dims,)
#       The eigenvalues of the Radius of Gyration tensor.
#     - R: float
#       The Radius of Gyration.
#     """
#     dims = traj.shape[1]  # Number of dimensions
#     T = np.zeros((dims, dims))  # Initialize the Radius of Gyration tensor

#     traj_mean = np.mean(traj, axis=0)  # Calculate the mean position

#     # Calculate each element of the RoG tensor
#     for i in range(dims):
#         for j in range(i, dims):
#             T[i, j] = np.mean((traj[:, i] - traj_mean[i]) * (traj[:, j] - traj_mean[j]))

#     # Fill in the lower triangle of the tensor (symmetric)
#     T = T + T.T - np.diag(T.diagonal())

#     # Calculate the eigenvectors and eigenvalues
#     eigenvalues, eigenvectors = np.linalg.eig(T)

#     # Ensure eigenvalues are real (sometimes they may have small imaginary parts due to numerical precision)
#     eigenvalues = np.real(eigenvalues)

#     # Ensure eigenvectors are real
#     eigenvectors = np.real(eigenvectors)

#     # Calculate the Radius of Gyration
#     R = np.sqrt(np.sum(eigenvalues))

#     return T, eigenvectors, eigenvalues, R


In [53]:
import numpy as np

def RadiusOfGyration(traj):
    """
    Calculate the Radius of Gyration tensor (T), eigenvectors (V), eigenvalues (lambda), 
    and Radius of Gyration (R) of a trajectory.

    Parameters:
    - traj: numpy.ndarray, shape (N, dims)
      Input trajectory data where N is the number of frames and dims is the number of dimensions.

    Returns:
    - T: numpy.ndarray, shape (dims, dims)
      The Radius of Gyration tensor.
    - V: numpy.ndarray, shape (dims, dims)
      The eigenvectors of the Radius of Gyration tensor.
    - lambda: numpy.ndarray, shape (dims,)
      The eigenvalues of the Radius of Gyration tensor.
    - R: float
      The Radius of Gyration.
    """
    dims = traj.shape[1]  # -1 was added before, Number of dimensions
    T = np.zeros((dims, dims))  # Initialize the Radius of Gyration tensor

    traj_mean = np.mean(traj, axis=0)  # Calculate the mean position

    # Calculate each element of the RoG tensor
    for i in range(dims):
        for j in range(i, dims):
            T[i, j] = np.mean((traj[:, i] - traj_mean[i]) * (traj[:, j] - traj_mean[j]))

    # Fill in the lower triangle of the tensor (symmetric)
    T = T + T.T - np.diag(T.diagonal())

    # Calculate the eigenvectors and eigenvalues
    eigenvalues, eigenvectors = np.linalg.eig(T)

    # Ensure eigenvalues are real (sometimes they may have small imaginary parts due to numerical precision)
    eigenvalues = np.real(eigenvalues)

    # Ensure eigenvectors are real
    eigenvectors = np.real(eigenvectors)

    # Calculate the Radius of Gyration
    R = np.sqrt(np.sum(eigenvalues))

    return T, eigenvectors, eigenvalues, R


FEATURE DEFINITION FUNCTIONS

In [1]:
def ExponentAlpha(traj):
    N = traj.shape[0]  # Number of points    #rows
    dims = traj.shape[1] - 1  # Number of dimensions   #columns
    pos = traj[:, :dims]

    maxlag = int(np.sqrt(N))
# \
    msd = MSD(pos, maxlag)
    msd_ = np.zeros_like(msd)

    for t in range(1, maxlag + 1):   ##
        dist_to_edge = min(t - 1, maxlag - t)
        smooth_len = min(2, dist_to_edge)
        lft = max(t - smooth_len, 1)
        rgt = min(t + smooth_len, maxlag)
        msd_[t - 1] = np.mean(msd[lft - 1:rgt + 1])    ## Added "-1" in the indexing for msd_ to account for zero indexing in python. same thing for [lft - 1:rgt + 1]

    dmsd = np.diff(np.log(msd_))
    dt = np.diff(np.log(np.arange(1, maxlag + 1)))

    alpha = np.mean(dmsd / dt)
    alpha = min(max(alpha, 0), 2)
    return alpha


In [38]:
def MSDRatio(traj):
    N = traj.shape[0]  # Number of points
    dims = traj.shape[1] - 1  # Number of dimensions
    pos = traj[:, :dims]  # Get only the position values, exclude time

    maxlag = round(N / 3)  # Max time lag

    # Evaluate the MSD
    msd = MSD(pos, maxlag)  # Assuming MSD function is defined elsewhere

    # Compare each possible time lag
    msdr = np.full((maxlag, maxlag), np.nan)
    for n1 in range(maxlag - 1):
        for n2 in range(n1 + 1, maxlag):
            msdr[n1, n2] = msd[n1] / msd[n2] - n1 / n2

    # Output
    msdr = np.nanmean(msdr)
    msdr = min(max(msdr, -1/3), 1)
    return msdr


In [39]:
def Gaussianity(traj):
    # Initialize
    N = traj.shape[0] - 1  # Number of steps
    dims = traj.shape[1] - 1  # Number of dimensions
    pos = traj[:, :dims]  # Get only the position values, exclude time

    maxlag = round(np.sqrt(N))  # Max time lag
    r2 = np.zeros(maxlag)
    r4 = np.zeros(maxlag)

    # Evaluate squared and quartic displacements
    for n in range(1, maxlag + 1):
        # Evaluate squared displacements
        sqdisp = np.sum((pos[n:] - pos[:-n])**2, axis=1)

        # Evaluate average squared & quartic displacements
        r2[n - 1] = np.mean(sqdisp)
        r4[n - 1] = np.mean(sqdisp**2)

    # Calculate Gaussianity measure
    g = np.mean((2 * r4) / (3 * r2**2) - 1)
    return g

In [40]:
def FractalDimension(traj):
    """
    Calculate the fractal dimension (Df) of a trajectory.

    Parameters:
    - traj: numpy.ndarray, shape (N, dims+1)
      Input trajectory data where N is the number of frames and dims is the number of dimensions.
      The last column is typically used for time or frame index.

    Returns:
    - Df: float
      The fractal dimension of the trajectory.
    """
    N = traj.shape[0] - 1
    dims = traj.shape[1] - 1
    pos = traj[:, :dims]

    # Evaluate all frame-to-frame displacements
    displacements = np.sqrt(np.sum((pos[1:] - pos[:-1])**2, axis=1))

    # Evaluate the largest distance between any two positions
    d_max = 0
    for i in range(N):
        dist = np.sqrt(np.max(np.sum((pos[i + 1:] - pos[i])**2, axis=1)))
        if dist > d_max:
            d_max = dist

    # Evaluate the total length of the path
    L = np.sum(displacements)

    # Calculate the fractal dimension
    Df = np.log(N) / np.log(N * d_max / L)
    Df = min(max(Df, 1), 4)
    
    return Df


In [58]:
def Asymmetry(traj):
    """
    Calculate the asymmetry measure for a trajectory.

    Parameters:
    - traj: numpy.ndarray, shape (N, dims+1)
      Input trajectory data where N is the number of frames and dims is the number of dimensions.
      The last column is typically used for time or frame index.

    Returns:
    - asym: float
      The asymmetry measure of the trajectory.
    """
    dims = traj.shape[1] - 1  # Number of dimensions
    pos = traj[:, :dims]  # Get only the position values, exclude time

    # Get the Radius of Gyration eigenvalues
    _, _, l, _ = RadiusOfGyration(pos)

    # Shorthand
    lxy = (l[0] - l[1]) / (l[0] + l[1])
    lxz = (l[0] - l[2]) / (l[0] + l[2])
    lyz = (l[1] - l[2]) / (l[1] + l[2])

    # Calculate the asymmetry measure
    asym = lxy**2 + lxz**2 + lyz**2

    return asym


In [42]:
def Kurtosis(traj):
    """
    Calculate the kurtosis measure for a trajectory.

    Parameters:
    - traj: numpy.ndarray, shape (N, dims+1)
      Input trajectory data where N is the number of frames and dims is the number of dimensions.
      The last column is typically used for time or frame index.

    Returns:
    - k: float
      The kurtosis measure of the trajectory.
    """
    dims = traj.shape[1]   # Number of dimensions
    pos = traj[:, :dims]  # Get only the position values, exclude time

    # Get the Radius of Gyration eigenvectors
    _, v, _, _ = RadiusOfGyration(pos)

    # Project the positions onto the dominant eigenvector (1-D result --> dot prod)
    proj = np.dot(pos, v[0])

    # Calculate kurtosis
    k = np.mean((proj - np.mean(proj))**4) / np.std(proj)**4
    return k

In [43]:
def Efficiency(traj):
    """
    Calculate the efficiency measure for a trajectory.

    Parameters:
    - traj: numpy.ndarray, shape (N, dims+1)
      Input trajectory data where N is the number of frames and dims is the number of dimensions.
      The last column is typically used for time or frame index.

    Returns:
    - e: float
      The efficiency measure of the trajectory.
    """
    N = traj.shape[0]  # Number of steps
    dims = traj.shape[1] - 1  # Number of dimensions
    pos = traj[:, :dims]  # Get only the position values, exclude time

    # Evaluate the ending squared displacement
    disp_end = np.sum((pos[-1] - pos[0])**2)

    # Evaluate the sum of frame-to-frame squared displacements
    disp_f2f = np.sum(np.sum((pos[1:] - pos[:-1])**2, axis=1))

    # Calculate efficiency
    e = disp_end / (N * disp_f2f)
    return e


In [44]:
def Straightness(traj):
    """
    Calculate the straightness measure for a trajectory.

    Parameters:
    - traj: numpy.ndarray, shape (N, dims+1)
      Input trajectory data where N is the number of frames and dims is the number of dimensions.
      The last column is typically used for time or frame index.

    Returns:
    - s: float
      The straightness measure of the trajectory.
    """
    dims = traj.shape[1] - 1  # Number of dimensions
    pos = traj[:, :dims]  # Get only the position values, exclude time

    # Evaluate the ending squared displacement
    disp_end = np.sqrt(np.sum((pos[-1] - pos[0])**2))

    # Evaluate the sum of frame-to-frame squared displacements
    disp_f2f = np.sum(np.sqrt(np.sum((pos[1:] - pos[:-1])**2, axis=1)))

    # Calculate straightness
    s = disp_end / disp_f2f
    return s


In [45]:
def Trappedness(traj):
    """
    Calculate the trappedness measure for a trajectory.

    Parameters:
    - traj: numpy.ndarray, shape (N, dims+1)
      Input trajectory data where N is the number of frames and dims is the number of dimensions.
      The last column is typically used for time or frame index.

    Returns:
    - Pt: float
      The trappedness measure of the trajectory.
    """
    N = traj.shape[0] - 1  # Number of steps
    dims = traj.shape[1] - 1  # Number of dimensions
    pos = traj[:, :dims]  # Get only the position values, exclude time

    # Evaluate the trapped time
    t = traj[-1, -1] - traj[0, -1]

    # Evaluate the maximum displacement
    r0 = 0
    for i in range(N):
        dist = np.sqrt(np.max(np.sum((pos[i + 1:] - pos[i])**2, axis=1)))
        if dist > r0:
            r0 = dist

    # Evaluate the short-time diffusion coefficient from the first two frames
    displacements_1 = np.sqrt(np.sum((pos[1:] - pos[:-1])**2, axis=1))
    displacements_2 = np.sqrt(np.sum((pos[2:] - pos[:-2])**2, axis=1))
    D = (np.mean(displacements_2) - np.mean(displacements_1)) / np.mean(np.diff(traj[:, -1]))

    # Calculate trappedness
    Pt = 1 - np.exp(0.2048 - 0.25117 * (4 * D * t / r0**2))
    # Pt = max(Pt, 0)  # Bound to [0, 1)
    Pt_max = np.max(Pt)  # Calculate the maximum value in the Pt array
    Pt_max = max(Pt_max, 0)  # Ensure that Pt_max is not negative

    
    return Pt_max


EXAMPLE USAGE

In [2]:
import scipy.io as spi
import imageio

class TIF_File:
    """
    This class reads a .tif file and stores the image data as a numpy array.
    """
    def __init__(self, filepath):
        self.filepath = filepath
        self.data = self._read_tif()

    def _read_tif(self):
        """
        Reads the .tif file and returns the image data as a numpy array.
        """
        return imageio.v2.imread(self.filepath)

    def get_data(self):
        """
        Returns the image data as a numpy array.
        """
        return self.data

In [11]:
tif = TIF_File("c:\\Users\\funke\\Documents\\WideField FWHM code\\TIF_Reader" + '\\' + "test_1.tif")
trajs = tif.get_data()
trajs.shape


(512, 512)


In [64]:
# import os
# os.chdir("c:\\Users\\funke\\Documents\\WideField FWHM code\\TIF_Reader")
# os.getcwd()

In [84]:
def RandForest_Features(trajs, wb=None, c=0, C=1):
    # Argument Defaults
    if wb is None:
        print("Extracting features... (00.00%)")

    # Initialize
    W, D, P = trajs.shape
    # W, P = trajs.shape
    ftrs = np.zeros((6, P))

    # Feature Extraction Loop
    for p in range(P): 
        # Temporary variable for convenience
        # traj = trajs[:, :, p]
        traj = trajs[:, p]

        # Don't do anything if there's nothing to do
        if np.all(np.isnan(traj)):
            continue

        # Only get the part that is not NaN
        traj = traj[~np.any(np.isnan(traj), axis=1), :]

        # Extract each feature in turn
        ftrs[0, p] = ExponentAlpha(traj)  # MSD based``
        ftrs[1, p] = MSDRatio(traj)
        ftrs[2, p] = FractalDimension(traj)  # Fractal based
        ftrs[3, p] = Asymmetry(traj)  # Radius of Gyration based
        ftrs[4, p] = Straightness(traj)  # Linearity based
        ftrs[5, p] = Trappedness(traj)

        # Update the user
        if (p + 1) % (P // 10) == 0:
            i=100 * (c + p + 1) / C
            print(f"Extracting features... ({i/1000:.2f}%)")

    # Cleanup
    if wb is None:
        print("Extraction complete 100.00%.")
        
    return ftrs, ftrs.shape

In [83]:
RandForest_Features(trajs)

Extracting features... (00.00%)


C:\Users\funke\AppData\Local\Temp\ipykernel_20832\2309831816.py:19: RuntimeWarning: overflow encountered in ushort_scalars
  t = traj[-1, -1] - traj[0, -1]


Extracting features... (5.10%)
Extracting features... (10.20%)
Extracting features... (15.30%)
Extracting features... (20.40%)
Extracting features... (25.50%)
Extracting features... (30.60%)
Extracting features... (35.70%)
Extracting features... (40.80%)
Extracting features... (45.90%)
Extracting features... (51.00%)
Extraction complete 100.00%.


(array([[4.76241065e-03, 0.00000000e+00, 6.82207789e-04, ...,
         0.00000000e+00, 1.49702524e-02, 6.61497510e-03],
        [5.29185567e-01, 5.28507410e-01, 5.29525281e-01, ...,
         5.28624499e-01, 5.29669336e-01, 5.30115578e-01],
        [4.00000000e+00, 4.00000000e+00, 4.00000000e+00, ...,
         4.00000000e+00, 4.00000000e+00, 4.00000000e+00],
        [4.86453339e-03, 2.46008557e-03, 4.02047153e-03, ...,
         6.95257884e-03, 1.76022223e-02, 7.67801449e-03],
        [9.52028232e-03, 1.00553971e-02, 1.01118442e-02, ...,
         9.75996314e-03, 9.82255022e-03, 9.66009101e-03],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]]),
 (6, 512))